In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import timeit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import statsmodels.api as sm


In [2]:
import os
os.getcwd()
os.chdir("/home/aumaron/Desktop/other_projects/")

In [3]:
# Data
rf = pd.read_excel("datasets/o_ring_failure_wiith_constant.xlsx", engine="openpyxl")
ring_failure = pd.read_excel("datasets/o_ring_failure.xlsx", engine="openpyxl")
# ring_failure.dropna(how="all", subset=["Target (Total orders)"], inplace=True)

# ring_failure.fillna(demand_forecast.mean(), inplace=True)

print(ring_failure.shape)
target = np.array(ring_failure.loc[:, "number_experiencing_thermal_distress"])
ring_failure.drop(columns=["number_experiencing_thermal_distress"], inplace=True)
train_array = np.array(ring_failure)

target1 = np.array(rf.loc[:, "number_experiencing_thermal_distress"])
rf.drop(columns=["number_experiencing_thermal_distress"], inplace=True)
train_array1 = np.array(rf)



(23, 4)


### Linear Model without intercept (without penalty)

In [ ]:
#Linear Regression sklearn
lm_sk = linear_model.LinearRegression(fit_intercept=False)  # Taking the first column with var=0 as the constant column
lm_sk.fit(train_array, target)
print(lm_sk.coef_)
print(lm_sk.intercept_)

In [ ]:
#Linear Regression sklearn
lm_sk = linear_model.LinearRegression(fit_intercept=False)  # Taking the first column with var=0 as the constant column
lm_sk.fit(train_array1, target1)
print(lm_sk.coef_)
print(lm_sk.intercept_, "\n")

#Linear Regression sklearn
lm_sk = linear_model.LinearRegression(fit_intercept=True)  # Taking the first column with var=0 as the constant column
lm_sk.fit(train_array1, target1)
print(lm_sk.coef_)
print(lm_sk.intercept_, "\n")


lm_sk = linear_model.LinearRegression(fit_intercept=True)  # Taking the first column with var=0 as the constant column
lm_sk.fit(train_array1[:, 1:4], target1)
print(lm_sk.coef_)
print(lm_sk.intercept_)

In [ ]:
# Linear regression statsmodel
import statsmodels.api as sm
# train_array = np.column_stack((train_array, np.ones(train_array.shape[0])))
train_array_new = sm.add_constant(train_array1[:, 1:4], has_constant="add")
lm_st = sm.OLS(target, train_array_new).fit()
print(lm_st.params)
lm_st.summary()

In [ ]:
# Linear regression statsmodel
import statsmodels.api as sm
# train_array = np.column_stack((train_array, np.ones(train_array.shape[0])))
lm_st = sm.OLS(target, train_array)
result = lm_st.fit()
print(result.params)
result.summary()

### Model with Intercept (without penalty)

In [56]:
#Linear Regression sklearn
lm_sk = linear_model.LinearRegression()
lm_sk.fit(train_array, target)
print(lm_sk.coef_)
print(lm_sk.intercept_)

[-0.05138594  0.00175701  0.01429284]
3.527093383307082


In [57]:
# Linear regression statsmodel
import statsmodels.api as sm
# train_array = np.column_stack((train_array, np.ones(train_array.shape[0])))
train_array_new = sm.add_constant(train_array, has_constant="add")
lm_st = sm.OLS(target, train_array_new).fit()
print(lm_st.params)
lm_st.summary()

[ 3.52709338e+00 -5.13859399e-02  1.75700897e-03  1.42928426e-02]


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     3.563
Date:                Mon, 03 May 2021   Prob (F-statistic):             0.0337
Time:                        23:38:11   Log-Likelihood:                -17.308
No. Observations:                  23   AIC:                             42.62
Df Residuals:                      19   BIC:                             47.16
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.5271      1.307      2.699      0.014       0.791       6.263
x1            -0.0514      0.018     -2.802      0.011      -0.090      -0.013
x2             0.0018      0.003      0.517      0.611      -0.005       0.009
x3             0.0143      0.035      0.407      0.689      -0.059       0.088
==============================================================================
Omnibus:                       17.300   Durbin-Watson:                   2.392
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               18.847
Skew:                           1.686   Prob(JB):                     8.08e-05
Kurtosis:                       5.881   Cond. No.                     1.98e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.98e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Model with intercept (lasso penalty)

In [4]:
train_array_new = sm.add_constant(train_array, has_constant="skip")

In [79]:
#Linear Regression sklearn
lm_lasso_sk = linear_model.Lasso(alpha=1, fit_intercept=False, positive=False)
lm_lasso_sk.fit(train_array_new, target)
print(lm_lasso_sk.coef_)
print(lm_lasso_sk.intercept_)

[ 0.         -0.00149013  0.00313455 -0.        ]
0.0


In [75]:
#Linear Regression sklearn
lm_elasti_sk = linear_model.ElasticNet(l1_ratio=1, alpha=1, fit_intercept=False, positive=True)
lm_elasti_sk.fit(train_array_new, target)
print(lm_elasti_sk.coef_)


[0.        0.        0.0025622 0.       ]


In [72]:
# Linear regression statsmodel
import statsmodels.api as sm
lm_st = sm.OLS(target, train_array_new)
result = lm_st.fit()
# print(result.params)
# result.summary()
# Lasso
results_fr = lm_st.fit_regularized(method="elastic_net", alpha=1, L1_wt=1, start_params=result.params)
# final = sm.regression.linear_model.OLSResults(lm_st,
#                                               results_fr.params, 
#                                               lm_st.normalized_cov_params)
print(results_fr.params)
# final.summary()

[0.        0.        0.0025622 0.       ]


##### 

In [ ]:
##### 

##### 